In [1]:
import pandas as pd
import numpy as np
import random
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from ObjectToObjectRecommender import ObjectToObjectRecommender as OTO

In [2]:
Tk().withdraw()

# def getPatronPath():
#     filename = askopenfilename()
#     return(filename)

# def getInventoryPath():
#     filename = askopenfilename()
#     return(filename)

# Remove to select folder or paste path below.
# patron_path = getPatronPath()
# inventory_pat = getInventoryPath()

patron_path = 'C:/Users/Ben/Desktop/HRP/Data/Clean_Data/Clean_Patron.csv'
inventory_path = 'C:/Users/Ben/Desktop/HRP/Data/Clean_Data/Clean_Inventory.csv'

patron_df = pd.read_csv(patron_path)
inv_df = pd.read_csv(inventory_path)

# Drop random columns
patron_df.drop(columns = 'Unnamed: 0', inplace = True)
inv_df.drop(columns = 'Unnamed: 0', inplace = True)

In [3]:
# Create Instances of an item to item and user to user comparators
iti = OTO(patron_df, 'Patron_ID', 'Item_ID')
ptp = OTO(patron_df, 'Item_ID', 'Patron_ID')

In [18]:
def get_patron_pairs(patron_id):
    '''
    Creates a list of positive patron and item pairs and an equally as large list of negative pairs
    '''
    pair_list = []

    # Get a list of pairs of patron, item that the patron has read.
    has_list = iti.objects_by_actor[patron_id]
    for item in has_list:
        pair_list.append([patron_id, item, 1])

    has_len = len(pair_list)

    # Get a list of pairs of patron, item that the patron hasn't read.
    hasnt_list = random.sample([x for x in iti.object_ids if x not in has_list], has_len)

    for item in hasnt_list:
        pair_list.append([patron_id, item, 0])

    return pair_list

In [19]:
def get_feat_vec(pair):
    '''
    Creates a list of selected features for any patron and item pair.
    '''
    feat_vec = []

    # User-Item pair (u, i)
    patron = pair[0]
    item = pair[1]

    # Popularity of i
    feat_vec.append(len(patron_df[patron_df['Item_ID'] == item]))

    # The similarity between i and the most similar book the user u has read
    max = 0
    items_read = iti.get_objects(patron)

    for item_id in items_read:
        sim_score = iti.jaccard(item, item_id)
        if sim_score > max:
            max = sim_score

    feat_vec.append(max)

    # The similarity between the user u and the most similar user who has read i
    max = 0
    patron_read = ptp.get_objects(item)

    for patron_id in patron_read:
        sim_score = ptp.jaccard(patron, patron_id)
        if sim_score > max:
            max = sim_score

    feat_vec.append(max)

    return feat_vec

In [41]:
# Create a user and item pairs for all users in the test DataFrame
pairs_lists = [get_patron_pairs(patron_id) for patron_id in set(iti.actor_ids)]

# unpack user and item pair list into the pair list and label list
pairs  = []
label_list = []

for pair_list in pairs_lists:
    for pair in pair_list:
        pairs.append(pair[:2])
        label_list.append(pair[2])

In [43]:
# Generate the feature vector for each pair
pair_feats = [get_feat_vec(pair) for pair in pairs]

In [44]:
# Split the feature vectors and labels into 75% for training an 25% for testing.
X_train, X_test, y_train, y_test = train_test_split(pair_feats, label_list, test_size=0.25, random_state=16)

In [45]:
# Initailize the model and fit to the training data.
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

LogisticRegression()

In [56]:
y_pred = logreg.predict(X_test)

accuracy =  1 - (y_test - y_pred).sum() / len(y_pred)

In [57]:
print(accuracy)

1.0


In [49]:
# from sklearn import metrics

# cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
# print(cnf_matrix)

# from sklearn.metrics import classification_report

# target_names = ['shouldn\'t checkout', 'should checkout']
# print(classification_report(y_test, y_pred, target_names=target_names))